Author: Pavel Khokhlov (pkhokhlo@alumni.cmu.edu)

# Imports

In [1]:
import copy
import gc
import seaborn as sb
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import random

gc.collect()

0

In [2]:
import pandas as pd
# good to never do chained assignment
pd.set_option('mode.chained_assignment', 'raise')
#pd.set_option('mode.chained_assignment', 'warn') # default
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('max_columns', 999)
pd.set_option('max_rows', 40)

In [3]:
import torch
from torch import distributions
from torch import nn
import torch.utils.data as utils

import sys
sys.path.append("pytorch-generative")
from pytorch_generative import nn as pg_nn
from pytorch_generative.models import base

In [4]:
random.seed(10)

###### static variables

can be turned into command line args for a python file

In [5]:
BATCH_SIZE = 32

In [6]:
MIN_TEST_DATE = '20060101'
MIN_YEAR = 2000
MAX_YEAR = 2008

In [7]:
DAV_DIRECTORY = 'dav_profiles'
SIZE_DIRECTORY = 'size_400km_mask'
RAD_DIRECTORY = 'size_and_radial_prof'
TC_DIRECTORY = 'size_and_radial_prof'

LOG_DIRECTORY = 'run_multiorb_size_mask/'

# size calculation function

In [8]:
def distance_from_coord(lat1, lat2, lon1, lon2):
    """
    Return distance in KM given a pair of lat-lon coordinates.
    :param lat1: Starting latitude
    :param lat2: Ending latitude
    :param lon1: Starting Longitude
    :param lon2: Ending longitude
    :return: Arc length of great circle
    """
    # https://www.movable-type.co.uk/scripts/latlong.html
    R = 6371
    latDel = lat1 - lat2
    lonDel = lon1 - lon2
    a = (
            np.sin(latDel / 2 * np.pi / 180) ** 2 +
            np.cos(lat1 * np.pi / 180) * np.cos(lat2 * np.pi / 180) *
            np.sin(lonDel / 2 * np.pi / 180) ** 2
    )
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def coord_from_distance(lat, lon, distance):
    """
    Find the bounding box for the stamp based on center coord and distance
    :param lat: Latitude of stamp center
    :param lon: Longitude of stamp center
    :param distance: Stamp radius
    :return: Dictionary of bounding latitudes and longitudes for stamp
    """
    # https://www.movable-type.co.uk/scripts/latlong.html
    R = 6371
    latEnd = [float()] * 360
    lonEnd = [float()] * 360
    bearings = range(0, 360)
    distance = float(distance)
    ii = 0
    for bearing in bearings:
        latEnd[ii] = np.arcsin(np.sin(lat * np.pi / 180) * np.cos(distance / R) +
                               np.cos(lat * np.pi / 180) * np.sin(distance / R) *
                               np.cos(bearing * np.pi / 180)
                               ) * 180 / np.pi
        lonEnd[ii] = lon + np.arctan2(
            np.sin(bearing * np.pi / 180) * np.sin(distance / R) * np.cos(lat * np.pi / 180),
            np.cos(distance / R) - np.sin(lat * np.pi / 180) * np.sin(latEnd[ii] * np.pi / 180)
        ) * 180 / np.pi
        ii += 1
    return ({
        'latHi': np.max(latEnd),
        'latLo': np.min(latEnd),
        'lonHi': np.max(lonEnd),
        'lonLo': np.min(lonEnd)
    })


stamp_area_cached = {}
def stamp_area(lat, distance=400, resolution=0.04):
    """
    Find the number of pixels in a stamp at a given latitude
    :param lat: Latitude of stamp center
    :param distance: Stamp radius
    :return: integer number of pixels in image
    """
    global stamp_area_cached
    if stamp_area_cached.get(lat):
        return stamp_area_cached.get(lat)
    bbox = coord_from_distance(lat, 0, distance)
    # Build the grid
    latpix = np.int(np.ceil((bbox['latHi'] - bbox['latLo']) / resolution))
    if latpix % 2 == 0:
        latpix += 1
    lonpix = np.int(np.ceil((bbox['lonHi'] - bbox['lonLo']) / resolution))
    if lonpix % 2 == 0:
        lonpix += 1
    lats = [resolution*(ii - np.floor(latpix/2)) for ii in range(latpix)]
    lons = [resolution*(ii - np.floor(lonpix/2)) for ii in range(lonpix)]
    grid = np.array(np.meshgrid(lats, lons)).reshape(2, latpix*lonpix).T
    # Compute radii over grid
    rads = distance_from_coord(lat * np.ones((grid.shape[0])),
                               lat * np.ones((grid.shape[0])) + grid[:, 0],
                               np.zeros((grid.shape[0])),
                               grid[:, 1])
    val = np.sum(rads < distance)
    stamp_area_cached[lat] = val
    return val

# Data loading

In [9]:
def get_orb_files(file_dir, location_prefix, orb_suffix, min_year=2000, max_year=2005):
    years = [str(i) for i in range(min_year, max_year + 1)]
    files = os.listdir(file_dir)
    rad_files = []
    for f in files:
        is_orb = orb_suffix in f
        year_valid = False
        location_valid = f.startswith(location_prefix)
        for year in years:
            year_valid = year_valid or (year in f)
        if is_orb and year_valid and location_valid:
            rad_files.append(f)
    return rad_files

In [10]:
#dav_files = get_orb_files(DAV_DIRECTORY, 'AL', '_DAV.csv', min_year=MIN_YEAR, max_year=MAX_YEAR)
#dacv_files = get_orb_files(DAV_DIRECTORY, 'AL', '_DACV.csv', min_year=MIN_YEAR, max_year=MAX_YEAR)
#darv_files = get_orb_files(DAV_DIRECTORY, 'AL', '_DARV.csv', min_year=MIN_YEAR, max_year=MAX_YEAR)
rad_files = get_orb_files(RAD_DIRECTORY, 'AL', '_rad.csv', min_year=MIN_YEAR, max_year=MAX_YEAR)
size_files = get_orb_files(SIZE_DIRECTORY, 'AL', '-size.csv', min_year=MIN_YEAR, max_year=MAX_YEAR)
tc_files = get_orb_files(TC_DIRECTORY, 'AL', '_TCdata.csv', min_year=MIN_YEAR, max_year=MAX_YEAR)

## size dataframe

In [11]:
#dfs = []
#for f in size_files:
#    path = f'/home/owner/Projects/tc/size_and_radial_prof/{f}'
#    storm_id = f.split('_')[0]
#    df = pd.read_csv(path, header=0, skiprows=[0, 2])
#    df.rename(columns={'temp_size': 'timestamp'}, inplace=True)
#    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round('min')
#    df['ID'] = storm_id
#    df.rename(columns=dict([(str(int(i)), str(i)) for i in range(-1, -100 - 1, -1)]), inplace=True)
#    df.sort_values('timestamp', inplace=True)
#    df.reset_index(drop=True, inplace=True)
#    dfs.append(df)
#df_size = pd.concat(dfs)

In [12]:
dfs = []
for f in size_files:
    path = f'{SIZE_DIRECTORY}/{f}'
    storm_id = f.split('-')[0]
    df = pd.read_csv(path, header=0, skiprows=[0, 2])
    del df['Unnamed: 1']
    del df['time']
    df = df.transpose()
    df.reset_index(inplace=True)
    # pandas columns can be both strings, floats, and ints - be careful
    df.rename(columns=dict([(i, str(int(i) - 100)) for i in range(100)]), inplace=True)
    df.rename(columns={'index': 'timestamp'}, inplace=True)
    df['ID'] = storm_id
    # round to minute for future joins - time series every 30 mins but actual timestamp sometimes off
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round('min')
    dfs.append(df)
df_size = pd.concat(dfs)

In [13]:
size_cols = [str(i) for i in range(-1, -100 - 1, -1)]

## RAD dataframe

In [14]:
dfs = []
for f in rad_files:
    path = f'{RAD_DIRECTORY}/{f}'
    storm_id = f.split('_')[0]
    df = pd.read_csv(path, header=0, skiprows=[0, 2])
    df.rename(columns={'radius': 'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round('min')
    df['ID'] = storm_id
    df.rename(columns=dict([(str(float(i)), str(i)) for i in range(5, 600 + 5, 5)]), inplace=True)
    df.sort_values('timestamp', inplace=True)
    df.reset_index(drop=True, inplace=True)
    dfs.append(df)
df_rad = pd.concat(dfs)

In [15]:
rad_cols = [str(i) for i in range(5, 600 + 5, 5)]

## TCdata dataframe

In [16]:
dfs = []
for f in tc_files:
    path = f'{TC_DIRECTORY}/{f}'
    storm_id = f.split('_')[0]
    df = pd.read_csv(path, header=0)
    df.rename(columns={'TIMESTAMP': 'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp']).dt.round('min')
    # interpolates latitude and wind speed between snapshots
    # snapshots not every 30 mins -> interpolate needed
    df = df.resample('0.5H', on='timestamp').mean().reset_index().interpolate()
    # round latitude to tenths place - used to compute storm image area
    df['LAT'] = df['LAT'].round(1)
    df['ID'] = storm_id
    dfs.append(df[['ID', 'timestamp', 'LAT', 'WIND']])
df_tc = pd.concat(dfs)

## dataframe merge

merge on timestamp and ID

In [17]:
df_size_rad = df_size.merge(df_rad, how='inner', on=['timestamp', 'ID'], suffixes=('_size', '_rad'))

In [18]:
df_size_rad_tc = df_size_rad.merge(df_tc, how='inner', on=['timestamp', 'ID'], suffixes=('', '_tc'))

In [19]:
storm_ids = list(df_size_rad_tc['ID'].unique())

## size normalization

In [20]:
area = df_size_rad_tc['LAT'].apply(stamp_area)

In [21]:
df_size_rad_tc['area'] = area

In [22]:
# normalize raw number pixels by image size
for size_col in size_cols:
    df_size_rad_tc[size_col] = df_size_rad_tc[size_col] / df_size_rad_tc['area']

# PCA size

In [23]:
pca_size = PCA(n_components=3)
# only 10% of data used in fitting
X_size_pca = df_size_rad_tc.loc[df_size_rad_tc['timestamp'] < MIN_TEST_DATE, size_cols].sample(frac=0.1).copy()
size_cols_means = X_size_pca.mean(0)
X_size_pca -= size_cols_means
#TODO: exclude null rows of rad in size PCA
pca_size.fit(X_size_pca)

# subtract mean from columns
size_pca_trans = pca_size.transform(df_size_rad_tc.loc[:, size_cols] - size_cols_means)

df_size_rad_tc['size_pca1'] = size_pca_trans[:, 0]
df_size_rad_tc['size_pca2'] = size_pca_trans[:, 1]
df_size_rad_tc['size_pca3'] = size_pca_trans[:, 2]

# PCA rad

In [24]:
pca_rad = PCA(n_components=2)
X_rad_pca = df_size_rad_tc.loc[df_size_rad_tc['timestamp'] < MIN_TEST_DATE, rad_cols].dropna().sample(frac=0.1).copy()
rad_cols_means = X_rad_pca.mean(0)
X_rad_pca -= rad_cols_means

pca_rad.fit(X_rad_pca)
# filter to exclude null rows
no_null_rows = np.array(df_size_rad_tc.loc[:, rad_cols].isna().sum(1) == 0)

rad_pca_trans = pca_rad.transform(df_size_rad_tc.loc[no_null_rows, rad_cols] - rad_cols_means)

df_size_rad_tc['rad_pca1'] = np.nan
df_size_rad_tc['rad_pca2'] = np.nan

df_size_rad_tc.loc[no_null_rows, 'rad_pca1'] = rad_pca_trans[:, 0]
df_size_rad_tc.loc[no_null_rows, 'rad_pca2'] = rad_pca_trans[:, 1]

# training tensor

In [25]:
# slices list will be used to build full dataset
slices_train = []
total = 0
size_nan = 0
for storm_id in storm_ids:
    if (df_size_rad_tc.loc[df_size_rad_tc['ID'] == storm_id, 'timestamp'] < MIN_TEST_DATE).sum() == 0:
        continue
    # sorted dataframe with just single storm
    storm_df = df_size_rad_tc.loc[df_size_rad_tc['ID'] == storm_id, :].sort_values('timestamp')
    
    # create numpy matrices and vectors of desired values
    size_mtx = storm_df[size_cols].values
    rad_mtx = storm_df[rad_cols].values
    
    wind_vec = storm_df['WIND'].values
    wind_diff_vec = storm_df['WIND'].diff().values
    
    # shift needed here because pca needs to be applied to past values
    rad_pca1_mtx = storm_df['rad_pca1'].shift(1).values
    rad_pca2_mtx = storm_df['rad_pca2'].shift(1).values
    
    size_pca1_mtx = storm_df['size_pca1'].shift(1).values
    size_pca2_mtx = storm_df['size_pca2'].shift(1).values
    size_pca3_mtx = storm_df['size_pca3'].shift(1).values
    
    # iterate over all 60 row images
    assert(rad_mtx.shape[0] == size_mtx.shape[0])
    for jj in range(size_mtx.shape[0] - 60):
        total += 1
        
        # create "slices" of 60 rows
        s_size = size_mtx[jj:(jj + 60)]
        s_rad = rad_mtx[jj:(jj + 60)]
        
        s_wind = wind_vec[jj:(jj + 60)]
        s_wind_diff = wind_diff_vec[jj:(jj + 60)]
        
        s_rad_pca1 = rad_pca1_mtx[jj:(jj + 60)]
        s_rad_pca2 = rad_pca2_mtx[jj:(jj + 60)]
        
        s_size_pca1 = size_pca1_mtx[jj:(jj + 60)]
        s_size_pca2 = size_pca2_mtx[jj:(jj + 60)]
        s_size_pca3 = size_pca3_mtx[jj:(jj + 60)]
        
        # exclude any images with nulls
        if(np.isnan(s_size).any()
           or np.isnan(s_rad).any()
           or np.isnan(s_wind).any()
           or np.isnan(s_wind_diff).any()
           or np.isnan(s_rad_pca1).any()
           or np.isnan(s_rad_pca2).any()
           or np.isnan(s_size_pca1).any()
           or np.isnan(s_size_pca2).any()
           or np.isnan(s_size_pca3).any()
          ):
            continue
        # element has 60 rows with ORBs and features used later for dataset construction
        slices_train.append([
                s_size,
                s_rad,
                s_wind,
                s_wind_diff,
                s_rad_pca1,
                s_rad_pca2,
                s_size_pca1,
                s_size_pca2,
                s_size_pca3,
        ])

In [26]:
## Stack all matrices together first, before splitting into training and test

size_np = np.stack(([el[0] for el in slices_train]), axis=0)
rad_np = np.stack(([el[1] for el in slices_train]), axis=0)

# We create a multi-channel matrix of the same size as size_np and rad_np
# in which we store the extra information we would like the last CNN to use for prediction

# many matrices of same dimension as size/rad but each row is same value
int_size_np = np.repeat(np.stack(([el[2] for el in slices_train]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
int_diff_size_np = np.repeat(np.stack(([el[3] for el in slices_train]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
rad_pca1_np = np.repeat(np.stack(([el[4] for el in slices_train]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
rad_pca2_np = np.repeat(np.stack(([el[5] for el in slices_train]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
extra_feats_size_np = np.stack((
    int_size_np, int_diff_size_np, rad_pca1_np, rad_pca2_np), axis=1)

int_rad_np = np.repeat(np.stack(([el[2] for el in slices_train]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
int_diff_rad_np = np.repeat(np.stack(([el[3] for el in slices_train]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
size_pca1_np = np.repeat(np.stack(([el[6] for el in slices_train]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
size_pca2_np = np.repeat(np.stack(([el[7] for el in slices_train]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
size_pca3_np = np.repeat(np.stack(([el[8] for el in slices_train]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
extra_feats_rad_np = np.stack((
    int_rad_np, int_diff_rad_np, size_pca1_np, size_pca2_np, size_pca3_np), axis=1)

print(size_np.shape, rad_np.shape)
print(int_size_np.shape, int_diff_size_np.shape, rad_pca1_np.shape, rad_pca2_np.shape)
print(extra_feats_size_np.shape)

print(int_rad_np.shape, int_diff_rad_np.shape, size_pca1_np.shape, size_pca2_np.shape, size_pca3_np.shape)
print(extra_feats_rad_np.shape)

# This dictionary will be important later to pass the arguments to the NN Architecture
dict_sizes_orb = {'channels_size':extra_feats_size_np.shape[1], 
                  'channels_rad':extra_feats_rad_np.shape[1]}
print(dict_sizes_orb)

(17051, 60, 100) (17051, 60, 120)
(17051, 60, 100) (17051, 60, 100) (17051, 60, 100) (17051, 60, 100)
(17051, 4, 60, 100)
(17051, 60, 120) (17051, 60, 120) (17051, 60, 120) (17051, 60, 120) (17051, 60, 120)
(17051, 5, 60, 120)
{'channels_size': 4, 'channels_rad': 5}


In [27]:
del slices_train
size_train_np = size_np.copy()
del size_np
rad_train_np = rad_np.copy()
del rad_np

In [28]:
extra_feats_size_train_np = extra_feats_size_np.copy()
del extra_feats_size_np
extra_feats_rad_train_np = extra_feats_rad_np.copy()
del extra_feats_rad_np

# test tensor

structure here same as training tensor

In [29]:
slices_test = []
total = 0
size_nan = 0
for storm_id in storm_ids:
    if (df_size_rad_tc.loc[df_size_rad_tc['ID'] == storm_id, 'timestamp'] < MIN_TEST_DATE).sum() == 0:
        storm_df = df_size_rad_tc.loc[df_size_rad_tc['ID'] == storm_id, :].sort_values('timestamp')

        size_mtx = storm_df[size_cols].values
        rad_mtx = storm_df[rad_cols].values

        wind_vec = storm_df['WIND'].values
        wind_diff_vec = storm_df['WIND'].diff().values
    
        rad_pca1_mtx = storm_df['rad_pca1'].shift(1).values
        rad_pca2_mtx = storm_df['rad_pca2'].shift(1).values

        size_pca1_mtx = storm_df['size_pca1'].shift(1).values
        size_pca2_mtx = storm_df['size_pca2'].shift(1).values
        size_pca3_mtx = storm_df['size_pca3'].shift(1).values

        assert(rad_mtx.shape[0] == size_mtx.shape[0])
        for jj in range(size_mtx.shape[0] - 60):
            total += 1
            s_size = size_mtx[jj:(jj + 60)]
            s_rad = rad_mtx[jj:(jj + 60)]

            s_wind = wind_vec[jj:(jj + 60)]
            s_wind_diff = wind_diff_vec[jj:(jj + 60)]

            s_rad_pca1 = rad_pca1_mtx[jj:(jj + 60)]
            s_rad_pca2 = rad_pca2_mtx[jj:(jj + 60)]

            s_size_pca1 = size_pca1_mtx[jj:(jj + 60)]
            s_size_pca2 = size_pca2_mtx[jj:(jj + 60)]
            s_size_pca3 = size_pca3_mtx[jj:(jj + 60)]

            if(np.isnan(s_size).any()
               or np.isnan(s_rad).any()
               or np.isnan(s_wind).any()
               or np.isnan(s_wind_diff).any()
               or np.isnan(s_rad_pca1).any()
               or np.isnan(s_rad_pca2).any()
               or np.isnan(s_size_pca1).any()
               or np.isnan(s_size_pca2).any()
               or np.isnan(s_size_pca3).any()
              ):
                continue
            slices_test.append([
                    s_size,
                    s_rad,
                    s_wind,
                    s_wind_diff,
                    s_rad_pca1,
                    s_rad_pca2,
                    s_size_pca1,
                    s_size_pca2,
                    s_size_pca3,
            ])

In [30]:
## Stack all matrices together first, before splitting into training and test

size_np = np.stack(([el[0] for el in slices_test]), axis=0)
rad_np = np.stack(([el[1] for el in slices_test]), axis=0)

# We create a multi-channel matrix of the same size as size_np and rad_np
# in which we store the extra information we would like the last CNN to use for prediction
# All columns in single row have same value (hence the np.repeat).

int_size_np = np.repeat(np.stack(([el[2] for el in slices_test]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
int_diff_size_np = np.repeat(np.stack(([el[3] for el in slices_test]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
rad_pca1_np = np.repeat(np.stack(([el[4] for el in slices_test]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
rad_pca2_np = np.repeat(np.stack(([el[5] for el in slices_test]), axis=0)[:, :, np.newaxis], size_np.shape[2], axis=2)
extra_feats_size_np = np.stack((
    int_size_np, int_diff_size_np, rad_pca1_np, rad_pca2_np), axis=1)

int_rad_np = np.repeat(np.stack(([el[2] for el in slices_test]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
int_diff_rad_np = np.repeat(np.stack(([el[3] for el in slices_test]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
size_pca1_np = np.repeat(np.stack(([el[6] for el in slices_test]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
size_pca2_np = np.repeat(np.stack(([el[7] for el in slices_test]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
size_pca3_np = np.repeat(np.stack(([el[8] for el in slices_test]), axis=0)[:, :, np.newaxis], rad_np.shape[2], axis=2)
extra_feats_rad_np = np.stack((
    int_rad_np, int_diff_rad_np, size_pca1_np, size_pca2_np, size_pca3_np), axis=1)

print(size_np.shape, rad_np.shape)
print(int_size_np.shape, int_diff_size_np.shape, rad_pca1_np.shape, rad_pca2_np.shape)
print(extra_feats_size_np.shape)

print(int_rad_np.shape, int_diff_rad_np.shape, size_pca1_np.shape, size_pca2_np.shape, size_pca3_np.shape)
print(extra_feats_rad_np.shape)

(6193, 60, 100) (6193, 60, 120)
(6193, 60, 100) (6193, 60, 100) (6193, 60, 100) (6193, 60, 100)
(6193, 4, 60, 100)
(6193, 60, 120) (6193, 60, 120) (6193, 60, 120) (6193, 60, 120) (6193, 60, 120)
(6193, 5, 60, 120)


In [31]:
del slices_test
size_test_np = size_np.copy()
del size_np
rad_test_np = rad_np.copy()
del rad_np

In [32]:
extra_feats_size_test_np = extra_feats_size_np.copy()
del extra_feats_size_np
extra_feats_rad_test_np = extra_feats_rad_np.copy()
del extra_feats_rad_np

In [33]:
print(size_train_np.shape, rad_train_np.shape, extra_feats_size_train_np.shape, extra_feats_rad_train_np.shape)
print(size_test_np.shape, rad_test_np.shape, extra_feats_size_test_np.shape, extra_feats_rad_test_np.shape)

(17051, 60, 100) (17051, 60, 120) (17051, 4, 60, 100) (17051, 5, 60, 120)
(6193, 60, 100) (6193, 60, 120) (6193, 4, 60, 100) (6193, 5, 60, 120)


# MultiOrb Torch Custom Dataset

In [34]:
class MultiOrbDataset(torch.utils.data.Dataset):
    '''Custom pytorch dataset with 4 separate tensors for size, rad, and their features 
    '''
    def __init__(self, size, rad, feats_size, feats_rad):
        self.size = torch.unsqueeze(size, dim=1)
        self.rad = torch.unsqueeze(rad, dim=1)
        self.feats_size = feats_size
        self.feats_rad = feats_rad

    def __getitem__(self, index):
        '''Returns the four matrices at index
        '''
        return self.size[index, :, :, :], self.rad[index, :, :, :], \
            self.feats_size[index, :, :, :], self.feats_rad[index, :, :, :]

    def __len__(self):
        return self.size.shape[0]

# construct train and test datasets from numpy tensors

train_dataset = MultiOrbDataset(
    size=torch.from_numpy(size_train_np.astype(np.float64)).type(torch.Tensor),
    rad=torch.from_numpy(rad_train_np.astype(np.float64)).type(torch.Tensor),
    feats_size=torch.from_numpy(extra_feats_size_train_np.astype(np.float64)).type(torch.Tensor),
    feats_rad=torch.from_numpy(extra_feats_rad_train_np.astype(np.float64)).type(torch.Tensor)
)
test_dataset = MultiOrbDataset(
    size=torch.from_numpy(size_test_np.astype(np.float64)).type(torch.Tensor),
    rad=torch.from_numpy(rad_test_np.astype(np.float64)).type(torch.Tensor),
    feats_size=torch.from_numpy(extra_feats_size_test_np.astype(np.float64)).type(torch.Tensor),
    feats_rad=torch.from_numpy(extra_feats_rad_test_np.astype(np.float64)).type(torch.Tensor)
)

# pytorch dataloader used during training to return batches
train_loader = utils.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = utils.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# manual garbage collection

python doesn't always free appropriately

In [35]:
del size_train_np, rad_train_np, extra_feats_size_train_np, extra_feats_rad_train_np

In [36]:
del size_test_np, rad_test_np, extra_feats_size_test_np, extra_feats_rad_test_np

In [37]:
# del size_np, rad_np, extra_feats_size_np, extra_feats_rad_np

In [38]:
del size_pca1_np, size_pca2_np, size_pca3_np, int_size_np, int_diff_size_np, rad_pca1_np, rad_pca2_np

In [39]:
gc.collect()

0

# Multi-head Model

In [40]:
import torch
from torch import distributions
from torch import nn

from pytorch_generative import nn as pg_nn
from pytorch_generative.models import base
from pytorch_generative.models.autoregressive.pixel_cnn import CausalResidualBlock


class PixelCNNMultiHead(base.AutoregressiveModel):
    """The PixelCNN model, initialized with multiple heads so to accommodate different
    orbs and other variabels."""

    def __init__(
        self,
        channels_size, channels_rad,
        in_channels=1,
        out_channels=1,
        n_residual=15,
        residual_channels=128,
        head_channels=32,
        sample_fn=None,
    ):
        """Initializes a new PixelCNN Multi Head instance.
        
        Architecture diagram: http://web.archive.org/web/20210604234735/https://miro.medium.com/max/911/1%2A3Fq4u66LYCLZElYuF_OOmA.png

        Args:
            in_channels: The number of input channels.
            out_channels: The number of output channels.
            n_residual: The number of residual blocks.
            residual_channels: The number of channels to use in the residual layers.
            head_channels: The number of channels to use in the two 1x1 convolutional
                layers at the head of the network.
            sample_fn: See the base class.
        """
        super().__init__(sample_fn)
        # masked convolutional layer
        self._input_h1 = pg_nn.CausalConv2d(
            mask_center=True,
            in_channels=in_channels,
            out_channels=2 * residual_channels,
            kernel_size=7,
            padding=3,
        )
        # residual blocks
        self._causal_layers_h1 = nn.ModuleList(
            [
                CausalResidualBlock(n_channels=2 * residual_channels)
                for _ in range(n_residual)
            ]
        )
        # two convolutional layers with two channel output
        self._head_h1 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(
                in_channels=2 * residual_channels,
                out_channels=head_channels,
                kernel_size=1,
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=head_channels, out_channels=out_channels, kernel_size=1
            )
        )
        
        # identical structure to above
        self._input_h2 = pg_nn.CausalConv2d(
            mask_center=True,
            in_channels=in_channels,
            out_channels=2 * residual_channels,
            kernel_size=7,
            padding=3,
        )
        self._causal_layers_h2 = nn.ModuleList(
            [
                CausalResidualBlock(n_channels=2 * residual_channels)
                for _ in range(n_residual)
            ]
        )
        self._head_h2 = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(
                in_channels=2 * residual_channels,
                out_channels=head_channels,
                kernel_size=1,
            ),
            nn.ReLU(),
            nn.Conv2d(
                in_channels=head_channels, out_channels=out_channels, kernel_size=1
            )
        )
        # final layer is 1x1 that uses two channels from initial layers and additional features
        self.cnn_size_final = nn.Conv2d(in_channels=out_channels + channels_size,
                                        out_channels=out_channels, kernel_size=1)
        self.cnn_rad_final = nn.Conv2d(in_channels=out_channels + channels_rad,
                                        out_channels=out_channels, kernel_size=1)

    def forward(self, size_orb, rad_orb, feats_size, feats_rad):
        '''Performs forward propagation of observations through network (i.e. prediction)
        '''
        # First propagate the first orb
        size_orb = self._input_h1(size_orb)
        for layer in self._causal_layers_h1:
            size_orb = size_orb + layer(size_orb)
        size_orb = self._head_h1(size_orb)
        
        # Then propagate the second orb
        rad_orb = self._input_h2(rad_orb)
        for layer in self._causal_layers_h2:
            rad_orb = rad_orb + layer(rad_orb)
        rad_orb = self._head_h2(rad_orb)
        
        # Attach all channels to both orbs
        size_orb_full = torch.cat((size_orb, feats_size), 1)
        rad_orb_full = torch.cat((rad_orb, feats_rad), 1)
        
        # Run the final convolutions
        size_preds = self.cnn_size_final(size_orb_full)
        rad_preds = self.cnn_rad_final(rad_orb_full)
        
        return size_preds, rad_preds

# New Trainer Class

#### Here we just inherit the Trainer class from the pytorch_generative model and change the necessary functions

In [41]:
import collections
import glob
import os
import re
import tempfile
import time

from torch.nn import parallel
from torch.utils import tensorboard
from pytorch_generative.trainer import Trainer


class TrainerMultiOrb(Trainer):
    """An object which encapsulates the training and evaluation loop, simply extending the Trainer class
    to MultiOrb architectures.
    """

    def __init__(
        self, model, loss_fn, optimizer, train_loader, eval_loader, lr_scheduler=None, clip_grad_norm=None,
        skip_grad_norm=None, sample_epochs=None, sample_fn=None, log_dir=None, save_checkpoint_epochs=1,
        n_gpus=0, device_id=None
    ):
        super().__init__(
            model=model,
            loss_fn=loss_fn,
            optimizer=optimizer,
            train_loader=train_loader,
            eval_loader=eval_loader,
            lr_scheduler=lr_scheduler,
            clip_grad_norm=clip_grad_norm,
            skip_grad_norm=skip_grad_norm,
            sample_epochs=sample_epochs,
            sample_fn=sample_fn,
            log_dir=log_dir,
            save_checkpoint_epochs=save_checkpoint_epochs,
            n_gpus=n_gpus,
            device_id=device_id
        )
    
    def train_one_batch(self, size_orb, rad_orb, feats_size, feats_rad):
        """Trains the model on a single batch of examples.

        Subclasses can override this method to define custom training loops.
        """
        
        size_preds, rad_preds = self.model(size_orb, rad_orb, feats_size, feats_rad)
        loss = self.loss_fn(size_orb, rad_orb, size_preds, rad_preds)
        return loss

    def _train_one_batch(self, size_orb, rad_orb, feats_size, feats_rad):
        self.model.train()
        size_orb, rad_orb, feats_size, feats_rad = size_orb.to(self.device), rad_orb.to(self.device), \
            feats_size.to(self.device), feats_rad.to(self.device)
        self.optimizer.zero_grad()
        loss = self._get_metrics_dict(self.train_one_batch(size_orb, rad_orb, feats_size, feats_rad))
        loss["loss"].backward()

        # uncomment to show gradient norm
        #total_norm = 0
        #for p in self.model.parameters():
        #    param_norm = p.grad.data.norm(2)
        #    total_norm += param_norm.item() ** 2
        #total_norm = total_norm ** (1. / 2)
        #print(f'norm: {total_norm:.4e}')

        norm = 0
        max_norm = self.clip_grad_norm or self.skip_grad_norm or None
        if max_norm:
            norm = torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm).item()
        self.optimizer.step()
        if self.lr_scheduler is not None:
            self.lr_scheduler.step()

        return {k: v.item() for k, v in loss.items()}

    def eval_one_batch(self, size_orb_eval, rad_orb_eval, feats_size_eval, feats_rad_eval):
        """Evaluates the model on a single batch of examples.

        Subclasses can override this method to define custom evaluation loops.
        """
        size_preds_eval, rad_preds_eval = self.model(size_orb_eval, rad_orb_eval, feats_size_eval, feats_rad_eval)
        loss = self.loss_fn(size_orb_eval, rad_orb_eval, size_preds_eval, rad_preds_eval)
        return loss

    def _eval_one_batch(self, size_orb_eval, rad_orb_eval, feats_size_eval, feats_rad_eval):
        with torch.no_grad():
            self.model.eval()
            size_orb_eval, rad_orb_eval, feats_size_eval, feats_rad_eval = size_orb_eval.to(self.device),\
                rad_orb_eval.to(self.device), feats_size_eval.to(self.device), feats_rad_eval.to(self.device)
            loss = self._get_metrics_dict(self.eval_one_batch(size_orb_eval, rad_orb_eval, feats_size_eval, feats_rad_eval))
            return {k: v.item() for k, v in loss.items()}

    def interleaved_train_and_eval(self, max_epochs, restore=True):
        """Trains and evaluates (after each epoch).

        Args:
            max_epochs: Maximum number of epochs to train for.
            restore: Wether to continue training from an existing checkpoint in
                self.log_dir.
        """
        if restore:
            try:
                self.restore_checkpoint()
            except FileNotFoundError:
                pass  # No checkpoint found in self.log_dir; train from scratch.

        for _ in range(max_epochs - self._epoch):
            start_time = time.time()
            # save model state at every epoch
            if self._epoch % 1 == 0:
                torch.save(self.model.state_dict(), self._path(f"model_state_{self._epoch}"))

            # Train.
            for i, (size_orb, rad_orb, feats_size, feats_rad) in enumerate(self.train_loader):
                self._examples_processed += size_orb.shape[0]
                lrs = {
                    f"group_{i}": param["lr"]
                    for i, param in enumerate(self.optimizer.param_groups)
                }
                self._summary_writer.add_scalars("loss/lr", lrs, self._step)
                
                loss = self._train_one_batch(size_orb, rad_orb, feats_size, feats_rad)
                self._log_metrics(loss, training=True)

                self._time_taken += time.time() - start_time
                start_time = time.time()
                self._summary_writer.add_scalar(
                    "speed/examples_per_sec",
                    self._examples_processed / self._time_taken,
                    self._step,
                )
                self._summary_writer.add_scalar(
                    "speed/millis_per_example",
                    self._time_taken / self._examples_processed * 1000,
                    self._step,
                )
                self._summary_writer.add_scalar("speed/epoch", self._epoch, self._step)
                self._summary_writer.add_scalar("speed/step", self._step, self._step)
                self._step += 1

            # Evaluate
            total_examples, total_loss = 0, collections.defaultdict(int)
            for (size_orb_eval, rad_orb_eval, feats_size_eval, feats_rad_eval) in self.eval_loader:
                n_examples = size_orb_eval.shape[0]
                total_examples += n_examples
                for key, loss in self._eval_one_batch(size_orb_eval, rad_orb_eval, feats_size_eval, feats_rad_eval).items():
                    total_loss[key] += loss * n_examples
            loss = {key: loss / total_examples for key, loss in total_loss.items()}
            self._log_metrics(loss, training=False)

            self._epoch += 1
            self._save_checkpoint()
            if self.sample_epochs and self._epoch % self.sample_epochs == 0:
                self.model.eval()
                with torch.no_grad():
                    tensor = self.sample_fn(self.model)
                self._summary_writer.add_images("sample", tensor, self._step)

        self._summary_writer.close()

# New Reproduce Function

In [42]:
def reproduce_multiorb(
    train_loader, test_loader, dict_sizes_orb=dict_sizes_orb,
    n_epochs=1000, batch_size=BATCH_SIZE, log_dir="/tmp/run", device="cpu", debug_loader=None
):
    """Training script with defaults to reproduce results.
    The code inside this function is self contained and can be used as a top level
    training script, e.g. by copy/pasting it into a Jupyter notebook.
    Args:
        n_epochs: Number of epochs to train for.
        batch_size: Batch size to use for training and evaluation.
        log_dir: Directory where to log trainer state and TensorBoard summaries.
        device: Device to train on (either 'cuda' or 'cpu').
        debug_loader: Debug DataLoader which replaces the default training and
            evaluation loaders if not 'None'. Do not use unless you're writing unit
            tests.
    """
    from torch import optim
    from torch.nn import functional as F
    from torch.optim import lr_scheduler

    from pytorch_generative import datasets
    from pytorch_generative import models
    
    # 4 channels: 2 channels for mu and sigma per orb
    out_channels = 2
    
    # Using our new multi head model, passing in the info about the different orb sizes
    # with the dict_sizes_orb dictionary
    model = PixelCNNMultiHead(
        in_channels=1,
        out_channels=out_channels,
        n_residual=15,
        residual_channels=16,
        head_channels=32,
        **dict_sizes_orb
    )
    # uncomment below to load a model and train starting from saved state
    #model.load_state_dict(torch.load(f'model_state'))
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[15 * len(train_loader), 250 * len(train_loader)], gamma=0.1)

    def loss_fn_multiorb(size_orb, rad_orb, size_preds, rad_preds, alpha=1.0, beta=1.0):
    
        # First compute the loss from the SIZE orb
        mean_channel_size = torch.div(size_preds[:, 0, :, :].float(), 1e4)
        std_channel_size_divided = torch.div(size_preds[:, 1, :, :].float(), 1e5)
        std_channel_size = torch.mul(torch.exp(std_channel_size_divided), 0.1)
        var_channel_size = torch.pow(std_channel_size, 2)

        # https://pytorch.org/docs/stable/generated/torch.nn.GaussianNLLLoss.html
        loss_size = nn.GaussianNLLLoss()
        output_size = loss_size(mean_channel_size, size_orb[:, 0, :, :], var_channel_size)

        # Then compute the loss from the RAD orb
        mean_channel_rad = torch.div(rad_preds[:, 0, :, :].float(), 1e4)
        std_channel_rad_divided = torch.div(rad_preds[:, 1, :, :].float(), 1e5)
        std_channel_rad = torch.mul(torch.exp(std_channel_rad_divided), 10)
        var_channel_rad = torch.pow(std_channel_rad, 2)

        loss_rad = nn.GaussianNLLLoss()
        output_rad = loss_rad(mean_channel_rad, rad_orb[:, 0, :, :], var_channel_rad)

        # Print average mean/variance/channel output from time to time
        # useful to keep track of exploding gradients/losses
        if np.random.uniform() < 0.1:
            print(f'size\tt: {torch.mean(size_orb[:, 0, :, :]):.2e}\tm: {torch.mean(mean_channel_size):.2e}\ts: {torch.mean(torch.sqrt(var_channel_size)):.2e}\tsch: {torch.mean(std_channel_size_divided):0.2f}')
            print(f'rad\tt: {torch.mean(rad_orb[:, 0, :, :]):.2e}\tm: {torch.mean(mean_channel_rad):.2e}\ts: {torch.mean(torch.sqrt(var_channel_rad)):.2e}\tsch: {torch.mean(std_channel_rad_divided):0.2f}')
            print('')

        return alpha * output_size + beta * output_rad
    
    trainer = TrainerMultiOrb(
        model=model,
        loss_fn=loss_fn_multiorb,
        optimizer=optimizer,
        train_loader=train_loader,
        eval_loader=test_loader,
        lr_scheduler=scheduler,
        log_dir=log_dir,
        #TODO: fix gpu device - it maps to cuda:0 and not what you specify
        device_id=device,
        n_gpus=1,
        skip_grad_norm=1e5,
    )
    trainer.interleaved_train_and_eval(n_epochs)
    return model

In [ ]:
model = reproduce_multiorb(train_loader=train_loader,
                           test_loader=test_loader,
                           log_dir=LOG_DIRECTORY,
                           device='cuda:2',
                           n_epochs=1000)

size	t: 2.44e-01	m: 3.58e-02	s: 1.01e-01	sch: 0.01
rad	t: -1.23e+01	m: -1.38e+00	s: 1.06e+01	sch: 0.06

size	t: 2.54e-01	m: 6.01e-02	s: 1.02e-01	sch: 0.02
rad	t: -1.36e+01	m: -2.06e+00	s: 1.13e+01	sch: 0.11

size	t: 2.88e-01	m: 1.00e-01	s: 1.03e-01	sch: 0.03
rad	t: -1.64e+01	m: -3.46e+00	s: 1.25e+01	sch: 0.20

size	t: 2.25e-01	m: 2.22e-01	s: 1.08e-01	sch: 0.07
rad	t: -9.75e+00	m: -1.00e+01	s: 6.01e+00	sch: -0.60

size	t: 2.23e-01	m: 2.15e-01	s: 1.07e-01	sch: 0.07
rad	t: -1.07e+01	m: -1.10e+01	s: 5.09e+00	sch: -0.89

size	t: 2.93e-01	m: 2.82e-01	s: 1.10e-01	sch: 0.09
rad	t: -1.66e+01	m: -1.55e+01	s: 5.17e+00	sch: -0.85

size	t: 2.88e-01	m: 2.92e-01	s: 1.10e-01	sch: 0.09
rad	t: -1.58e+01	m: -1.59e+01	s: 5.05e+00	sch: -0.83

size	t: 2.80e-01	m: 2.78e-01	s: 1.09e-01	sch: 0.08
rad	t: -1.57e+01	m: -1.60e+01	s: 5.35e+00	sch: -0.77

size	t: 2.48e-01	m: 2.57e-01	s: 1.08e-01	sch: 0.08
rad	t: -1.28e+01	m: -1.26e+01	s: 5.10e+00	sch: -0.85

size	t: 2.26e-01	m: 2.21e-01	s: 1.07e-01	sch: 0.06
rad	t: 

size	t: 2.79e-01	m: 2.42e-01	s: 5.90e-02	sch: -0.53
rad	t: -1.56e+01	m: -1.54e+01	s: 1.71e+00	sch: -1.91

size	t: 2.51e-01	m: 2.55e-01	s: 5.83e-02	sch: -0.54
rad	t: -1.34e+01	m: -1.29e+01	s: 1.48e+00	sch: -2.05

size	t: 3.09e-01	m: 3.32e-01	s: 3.84e-02	sch: -0.98
rad	t: -1.78e+01	m: -1.70e+01	s: 1.84e+00	sch: -1.97

size	t: 2.26e-01	m: 1.94e-01	s: 3.27e-02	sch: -1.14
rad	t: -1.07e+01	m: -9.88e+00	s: 1.76e+00	sch: -1.91

size	t: 2.79e-01	m: 2.31e-01	s: 3.63e-02	sch: -1.03
rad	t: -1.55e+01	m: -1.56e+01	s: 2.04e+00	sch: -1.83

size	t: 2.47e-01	m: 2.39e-01	s: 3.97e-02	sch: -0.94
rad	t: -1.30e+01	m: -1.19e+01	s: 2.18e+00	sch: -1.71

size	t: 3.21e-01	m: 3.65e-01	s: 4.00e-02	sch: -0.93
rad	t: -1.90e+01	m: -1.87e+01	s: 1.52e+00	sch: -1.99

size	t: 2.73e-01	m: 3.11e-01	s: 4.01e-02	sch: -0.93
rad	t: -1.54e+01	m: -1.53e+01	s: 1.67e+00	sch: -1.92

size	t: 2.53e-01	m: 2.85e-01	s: 3.88e-02	sch: -0.96
rad	t: -1.30e+01	m: -1.32e+01	s: 1.16e+00	sch: -2.36

size	t: 2.19e-01	m: 1.95e-01	s: 3.82e-02	sch: 

size	t: 2.57e-01	m: 2.62e-01	s: 1.37e-02	sch: -2.04
rad	t: -1.29e+01	m: -1.29e+01	s: 9.42e-01	sch: -2.54

size	t: 2.73e-01	m: 2.56e-01	s: 1.39e-02	sch: -2.02
rad	t: -1.46e+01	m: -1.46e+01	s: 9.33e-01	sch: -2.56

size	t: 2.68e-01	m: 2.55e-01	s: 1.47e-02	sch: -1.97
rad	t: -1.39e+01	m: -1.39e+01	s: 9.55e-01	sch: -2.52

size	t: 2.92e-01	m: 2.99e-01	s: 1.51e-02	sch: -1.93
rad	t: -1.65e+01	m: -1.65e+01	s: 9.41e-01	sch: -2.54

size	t: 2.59e-01	m: 2.45e-01	s: 1.51e-02	sch: -1.93
rad	t: -1.35e+01	m: -1.35e+01	s: 9.34e-01	sch: -2.57

size	t: 2.14e-01	m: 2.17e-01	s: 1.51e-02	sch: -1.94
rad	t: -8.92e+00	m: -8.92e+00	s: 8.90e-01	sch: -2.63

size	t: 2.59e-01	m: 2.44e-01	s: 1.47e-02	sch: -1.97
rad	t: -1.26e+01	m: -1.26e+01	s: 9.16e-01	sch: -2.57

size	t: 2.15e-01	m: 2.21e-01	s: 1.44e-02	sch: -1.99
rad	t: -9.57e+00	m: -9.52e+00	s: 9.31e-01	sch: -2.56

size	t: 2.72e-01	m: 2.59e-01	s: 1.43e-02	sch: -1.99
rad	t: -1.52e+01	m: -1.52e+01	s: 9.35e-01	sch: -2.53

size	t: 2.47e-01	m: 2.52e-01	s: 1.42e-02	sch: 

size	t: 2.53e-01	m: 2.61e-01	s: 2.18e-02	sch: -1.54
rad	t: -1.32e+01	m: -1.31e+01	s: 7.86e-01	sch: -2.70

size	t: 2.22e-01	m: 1.93e-01	s: 2.11e-02	sch: -1.57
rad	t: -9.51e+00	m: -9.48e+00	s: 7.69e-01	sch: -2.74

size	t: 2.36e-01	m: 2.20e-01	s: 2.12e-02	sch: -1.57
rad	t: -1.13e+01	m: -1.14e+01	s: 7.90e-01	sch: -2.72

size	t: 2.15e-01	m: 2.28e-01	s: 2.09e-02	sch: -1.58
rad	t: -8.08e+00	m: -8.20e+00	s: 7.85e-01	sch: -2.75

size	t: 2.74e-01	m: 2.53e-01	s: 2.12e-02	sch: -1.57
rad	t: -1.53e+01	m: -1.51e+01	s: 8.62e-01	sch: -2.60

size	t: 2.82e-01	m: 2.91e-01	s: 2.08e-02	sch: -1.59
rad	t: -1.55e+01	m: -1.55e+01	s: 8.74e-01	sch: -2.60

size	t: 2.51e-01	m: 2.34e-01	s: 2.16e-02	sch: -1.55
rad	t: -1.18e+01	m: -1.15e+01	s: 7.87e-01	sch: -2.73

size	t: 2.36e-01	m: 2.20e-01	s: 2.18e-02	sch: -1.54
rad	t: -1.11e+01	m: -1.11e+01	s: 7.52e-01	sch: -2.75

size	t: 2.66e-01	m: 2.72e-01	s: 2.13e-02	sch: -1.56
rad	t: -1.36e+01	m: -1.36e+01	s: 9.07e-01	sch: -2.59

size	t: 2.34e-01	m: 2.38e-01	s: 2.10e-02	sch: 

size	t: 2.89e-01	m: 2.98e-01	s: 1.34e-02	sch: -2.03
rad	t: -1.58e+01	m: -1.58e+01	s: 8.41e-01	sch: -2.64

size	t: 2.93e-01	m: 2.81e-01	s: 1.34e-02	sch: -2.03
rad	t: -1.62e+01	m: -1.62e+01	s: 8.49e-01	sch: -2.63

size	t: 2.59e-01	m: 2.67e-01	s: 1.28e-02	sch: -2.08
rad	t: -1.42e+01	m: -1.42e+01	s: 7.61e-01	sch: -2.72

size	t: 2.96e-01	m: 2.82e-01	s: 1.23e-02	sch: -2.12
rad	t: -1.60e+01	m: -1.60e+01	s: 7.73e-01	sch: -2.74

size	t: 2.47e-01	m: 2.54e-01	s: 1.16e-02	sch: -2.17
rad	t: -1.22e+01	m: -1.22e+01	s: 7.81e-01	sch: -2.74

size	t: 2.43e-01	m: 2.30e-01	s: 1.16e-02	sch: -2.18
rad	t: -1.15e+01	m: -1.16e+01	s: 7.52e-01	sch: -2.77

size	t: 2.62e-01	m: 2.66e-01	s: 1.14e-02	sch: -2.19
rad	t: -1.32e+01	m: -1.33e+01	s: 7.77e-01	sch: -2.74

size	t: 2.26e-01	m: 2.09e-01	s: 1.14e-02	sch: -2.19
rad	t: -1.02e+01	m: -1.02e+01	s: 7.78e-01	sch: -2.73

size	t: 2.59e-01	m: 2.44e-01	s: 1.15e-02	sch: -2.19
rad	t: -1.41e+01	m: -1.40e+01	s: 7.64e-01	sch: -2.73

size	t: 2.53e-01	m: 2.37e-01	s: 1.14e-02	sch: 

...

size	t: 2.55e-01	m: 2.54e-01	s: 1.48e-03	sch: -5.06
rad	t: -1.33e+01	m: -1.33e+01	s: 6.08e-01	sch: -2.99

size	t: 2.54e-01	m: 2.54e-01	s: 1.49e-03	sch: -5.02
rad	t: -1.31e+01	m: -1.31e+01	s: 6.16e-01	sch: -2.99

size	t: 2.51e-01	m: 2.51e-01	s: 1.47e-03	sch: -5.01
rad	t: -1.28e+01	m: -1.28e+01	s: 6.44e-01	sch: -2.94

size	t: 2.82e-01	m: 2.82e-01	s: 1.53e-03	sch: -4.98
rad	t: -1.62e+01	m: -1.62e+01	s: 6.23e-01	sch: -2.95

size	t: 2.79e-01	m: 2.79e-01	s: 1.49e-03	sch: -4.98
rad	t: -1.50e+01	m: -1.50e+01	s: 6.03e-01	sch: -2.98

size	t: 2.44e-01	m: 2.44e-01	s: 1.41e-03	sch: -5.07
rad	t: -1.19e+01	m: -1.19e+01	s: 6.22e-01	sch: -2.98

size	t: 2.46e-01	m: 2.45e-01	s: 1.46e-03	sch: -5.06
rad	t: -1.28e+01	m: -1.28e+01	s: 6.17e-01	sch: -2.98

size	t: 2.76e-01	m: 2.76e-01	s: 1.52e-03	sch: -4.99
rad	t: -1.52e+01	m: -1.52e+01	s: 6.21e-01	sch: -2.97

size	t: 2.88e-01	m: 2.88e-01	s: 1.55e-03	sch: -4.93
rad	t: -1.59e+01	m: -1.59e+01	s: 6.10e-01	sch: -2.99

size	t: 2.65e-01	m: 2.65e-01	s: 1.48e-03	sch: 

size	t: 2.77e-01	m: 2.77e-01	s: 1.51e-03	sch: -5.00
rad	t: -1.52e+01	m: -1.52e+01	s: 6.07e-01	sch: -2.98

size	t: 2.65e-01	m: 2.65e-01	s: 1.49e-03	sch: -5.00
rad	t: -1.50e+01	m: -1.50e+01	s: 6.01e-01	sch: -3.00

size	t: 2.89e-01	m: 2.89e-01	s: 1.50e-03	sch: -4.96
rad	t: -1.50e+01	m: -1.50e+01	s: 6.11e-01	sch: -2.98

size	t: 2.40e-01	m: 2.40e-01	s: 1.37e-03	sch: -5.12
rad	t: -1.11e+01	m: -1.11e+01	s: 5.97e-01	sch: -3.03

size	t: 2.36e-01	m: 2.36e-01	s: 1.40e-03	sch: -5.12
rad	t: -1.08e+01	m: -1.08e+01	s: 6.05e-01	sch: -3.02

size	t: 2.15e-01	m: 2.15e-01	s: 1.30e-03	sch: -5.21
rad	t: -8.20e+00	m: -8.20e+00	s: 5.90e-01	sch: -3.04

size	t: 2.22e-01	m: 2.22e-01	s: 1.31e-03	sch: -5.18
rad	t: -9.00e+00	m: -9.00e+00	s: 5.90e-01	sch: -3.04

size	t: 2.71e-01	m: 2.71e-01	s: 1.50e-03	sch: -4.98
rad	t: -1.43e+01	m: -1.43e+01	s: 6.24e-01	sch: -2.96

size	t: 2.82e-01	m: 2.82e-01	s: 1.53e-03	sch: -4.95
rad	t: -1.52e+01	m: -1.52e+01	s: 5.90e-01	sch: -3.02

size	t: 2.89e-01	m: 2.89e-01	s: 1.55e-03	sch: 

size	t: 2.81e-01	m: 2.81e-01	s: 1.51e-03	sch: -5.01
rad	t: -1.55e+01	m: -1.55e+01	s: 6.08e-01	sch: -3.00

size	t: 2.60e-01	m: 2.61e-01	s: 1.49e-03	sch: -5.06
rad	t: -1.36e+01	m: -1.36e+01	s: 5.90e-01	sch: -3.03

size	t: 2.37e-01	m: 2.37e-01	s: 1.41e-03	sch: -5.09
rad	t: -1.12e+01	m: -1.12e+01	s: 6.14e-01	sch: -3.01

size	t: 2.57e-01	m: 2.57e-01	s: 1.40e-03	sch: -5.09
rad	t: -1.28e+01	m: -1.28e+01	s: 5.82e-01	sch: -3.04

size	t: 2.45e-01	m: 2.45e-01	s: 1.41e-03	sch: -5.07
rad	t: -1.25e+01	m: -1.25e+01	s: 6.21e-01	sch: -2.98

size	t: 2.56e-01	m: 2.55e-01	s: 1.45e-03	sch: -5.03
rad	t: -1.26e+01	m: -1.26e+01	s: 6.01e-01	sch: -3.00

size	t: 2.43e-01	m: 2.43e-01	s: 1.44e-03	sch: -5.08
rad	t: -1.20e+01	m: -1.20e+01	s: 6.54e-01	sch: -2.94

size	t: 2.56e-01	m: 2.56e-01	s: 1.50e-03	sch: -5.04
rad	t: -1.22e+01	m: -1.22e+01	s: 6.80e-01	sch: -2.88

size	t: 2.49e-01	m: 2.49e-01	s: 1.44e-03	sch: -5.03
rad	t: -1.16e+01	m: -1.16e+01	s: 5.98e-01	sch: -3.02

size	t: 2.48e-01	m: 2.48e-01	s: 1.41e-03	sch: 

size	t: 2.90e-01	m: 2.90e-01	s: 1.58e-03	sch: -4.94
rad	t: -1.68e+01	m: -1.68e+01	s: 6.06e-01	sch: -2.98

size	t: 2.70e-01	m: 2.70e-01	s: 1.50e-03	sch: -5.01
rad	t: -1.37e+01	m: -1.37e+01	s: 6.26e-01	sch: -2.97

size	t: 2.22e-01	m: 2.22e-01	s: 1.39e-03	sch: -5.11
rad	t: -1.06e+01	m: -1.06e+01	s: 6.45e-01	sch: -2.94

size	t: 2.66e-01	m: 2.66e-01	s: 1.47e-03	sch: -5.01
rad	t: -1.44e+01	m: -1.44e+01	s: 6.11e-01	sch: -3.01

size	t: 2.83e-01	m: 2.83e-01	s: 1.49e-03	sch: -4.99
rad	t: -1.59e+01	m: -1.59e+01	s: 6.05e-01	sch: -2.99

size	t: 2.45e-01	m: 2.44e-01	s: 1.37e-03	sch: -5.08
rad	t: -1.13e+01	m: -1.13e+01	s: 5.83e-01	sch: -3.04

size	t: 2.71e-01	m: 2.71e-01	s: 1.49e-03	sch: -5.02
rad	t: -1.42e+01	m: -1.42e+01	s: 6.22e-01	sch: -2.98

size	t: 2.67e-01	m: 2.67e-01	s: 1.46e-03	sch: -5.02
rad	t: -1.52e+01	m: -1.52e+01	s: 6.07e-01	sch: -2.98

size	t: 2.59e-01	m: 2.59e-01	s: 1.48e-03	sch: -5.02
rad	t: -1.37e+01	m: -1.37e+01	s: 6.13e-01	sch: -2.98

size	t: 2.64e-01	m: 2.64e-01	s: 1.47e-03	sch: 

size	t: 2.56e-01	m: 2.56e-01	s: 1.43e-03	sch: -5.05
rad	t: -1.35e+01	m: -1.35e+01	s: 5.86e-01	sch: -3.04

size	t: 3.00e-01	m: 3.00e-01	s: 1.55e-03	sch: -4.98
rad	t: -1.66e+01	m: -1.66e+01	s: 5.98e-01	sch: -3.01

size	t: 2.45e-01	m: 2.45e-01	s: 1.41e-03	sch: -5.10
rad	t: -1.17e+01	m: -1.17e+01	s: 6.05e-01	sch: -2.99

size	t: 2.70e-01	m: 2.70e-01	s: 1.51e-03	sch: -5.00
rad	t: -1.45e+01	m: -1.45e+01	s: 6.05e-01	sch: -2.99

size	t: 2.28e-01	m: 2.28e-01	s: 1.39e-03	sch: -5.09
rad	t: -1.05e+01	m: -1.06e+01	s: 6.17e-01	sch: -2.98

